Упрощенный пример .jpeg

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image

Светимость Y пропорциональна мощности источника света (соотносится с восприятием яркости глазом человека).

Определяется как $\frac{77}{256} R + \frac{150}{256} G + \frac{29}{256} R$

$C_b = B - Y$ (хроматический синий), $C_r = R - Y$

In [2]:
y_quant = np.array([
[16, 11, 10, 16, 24, 40, 51, 61,],
[12, 12, 14, 19, 26, 58, 60, 55,],
[14, 13, 16, 24, 40, 57, 69, 56,],
[14, 17, 22, 29, 51, 87, 80, 62,],
[18, 22, 37, 56, 68, 109, 103, 77,],
[24, 35, 55, 64, 81, 104, 113, 92,],
[49, 64, 78, 87, 103, 121, 120, 101,],
[72, 92, 95, 98, 112, 100, 103, 99,],
])
chroma_quant = np.array([
[17, 18, 24, 47, 99, 99, 99, 99,],
[18, 21, 26, 66, 99, 99, 99, 99,],
[24, 26, 56, 99, 99, 99, 99, 99,],
[47, 66, 99, 99, 99, 99, 99, 99,],
[99, 99, 99, 99, 99, 99, 99, 99,],
[99, 99, 99, 99, 99, 99, 99, 99,],
[99, 99, 99, 99, 99, 99, 99, 99,],
[99, 99, 99, 99, 99, 99, 99, 99,],
])

Простой пример

In [3]:
import scipy.fft

a_orig = np.array([
    [52, 55, 61, 66, 70, 61, 64, 73,],
    [63, 59, 66, 90, 109, 85, 69, 72,],
    [62, 59, 68, 113, 144, 104, 66, 73,],
    [63, 58, 71, 122, 154, 106, 70, 69,],
    [67, 61, 68, 104, 126, 88, 68, 70,],
    [79, 65, 60, 70, 77, 68, 58, 75,],
    [85, 71, 64, 59, 55, 61, 65, 83,],
    [87, 79, 69, 68, 65, 76, 78, 94,],
])
# Сдвигаем диапазон значений
a_encode = a_orig - 128
# Находим дискретное косинусное преобразование для блока 8x8
a_encode = np.round(scipy.fft.dctn(a_encode, norm='ortho'))
print(a_encode)


[[-414.  -29.  -62.   25.   55.  -20.   -1.    2.]
 [   6.  -21.  -62.    8.   12.   -7.   -6.    7.]
 [ -46.    8.   77.  -26.  -30.   10.    6.   -5.]
 [ -49.   12.   34.  -14.  -10.    6.    1.    1.]
 [  11.   -8.  -12.   -2.   -1.    1.   -5.    2.]
 [ -10.    1.    3.   -3.   -0.    0.    2.   -0.]
 [  -3.   -1.    1.    0.    1.   -4.    2.   -3.]
 [  -1.   -1.   -0.   -3.   -0.   -0.   -1.    0.]]


In [4]:
# Квантование
a_encode = np.round(a_encode/y_quant).astype(int)
print(a_encode)

[[-26  -3  -6   2   2   0   0   0]
 [  0  -2  -4   0   0   0   0   0]
 [ -3   1   5  -1  -1   0   0   0]
 [ -4   1   2   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]]


In [5]:
def zigzag_order(matrix):
    rows, cols = matrix.shape
    result = []
    
    for s in range(rows + cols - 1):
        if s % 2 == 0:
            # Even index sum — go up-right
            for i in range(s, -1, -1):
                j = s - i
                if i < rows and j < cols:
                    result.append(matrix[i][j])
        else:
            # Odd index sum — go down-left
            for j in range(s, -1, -1):
                i = s - j
                if i < rows and j < cols:
                    result.append(matrix[i][j])
    
    return np.array(result)

# Переупорядочим элементы в соответствии с зигзаг-преобразованием
a_encode = zigzag_order(a_encode)
print(a_encode)

[-26  -3   0  -3  -2  -6   2  -4   1  -4   1   1   5   0   2   0   0  -1
   2   0   0   0   0   0   0  -1   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [6]:
# Example Huffman coding implementation from
# https://gist.github.com/mreid/fdf6353ec39d050e972b

def huffman(p):
    '''Return a Huffman code for an ensemble with distribution p.'''
    assert (sum(p.values()) == 1.0) # Ensure probabilities sum to 1

    # Base case of only two symbols, assign 0 or 1 arbitrarily
    if(len(p) == 2):
        return dict(list(zip(list(sorted(p.keys())), ['0', '1'])))

    # Create a new distribution by merging lowest prob. pair
    p_prime = p.copy()
    a1, a2 = lowest_prob_pair(p)
    p1, p2 = p_prime.pop(a1), p_prime.pop(a2)
    p_prime[a1 + a2] = p1 + p2

    # Recurse and construct code on new distribution
    c = huffman(p_prime)
    ca1a2 = c.pop(a1 + a2)
    c[a1], c[a2] = ca1a2 + '0', ca1a2 + '1'

    return c

def lowest_prob_pair(p):
    '''Return pair of symbols from distribution p with lowest probabilities.'''
    assert(len(p) >= 2) # Ensure there are at least 2 symbols in the dist.

    sorted_p = sorted(list(p.items()), key=lambda i_pi: (i_pi[1],i_pi[0]))
    return sorted_p[0][0], sorted_p[1][0]

# ex2 = { 'a': 0.25, 'b': 0.25, 'c': 0.2, 'd': 0.15, 'e': 0.15 }
# huffman(ex2)  # => {'a': '01', 'c': '00', 'b': '10', 'e': '110', 'd': '111'}


In [7]:
from typing import Counter
# Найдем частоту появления символов в изображении
a_cnt = dict(Counter(a_encode))

print(a_cnt)
# Нормировка
a_probs = {}
for key in a_cnt:
    a_probs[str(key)] = a_cnt[key]/64

print(a_probs)

{np.int64(-26): 1, np.int64(-3): 2, np.int64(0): 48, np.int64(-2): 1, np.int64(-6): 1, np.int64(2): 3, np.int64(-4): 2, np.int64(1): 3, np.int64(5): 1, np.int64(-1): 2}
{'-26': 0.015625, '-3': 0.03125, '0': 0.75, '-2': 0.015625, '-6': 0.015625, '2': 0.046875, '-4': 0.03125, '1': 0.046875, '5': 0.015625, '-1': 0.03125}


In [8]:
# Кодирование Хаффмана
a_huff_encode = huffman(a_probs)
print(a_huff_encode)
# Для удобства декодирования такой же словарь, но поменяли местами key и value
a_huff_decode = {v: k for k, v in a_huff_encode.items()}

{'0': '0', '2': '100', '1': '1111', '-3': '1100', '-4': '1101', '-1': '1010', '-6': '11100', '5': '11101', '-2': '10110', '-26': '10111'}


In [9]:
# Выходные биты
output_bits = []
for elem in a_encode:
    output_bits.append(a_huff_encode[str(elem)])

len(''.join(output_bits))

113

In [10]:
# Декодирование
a_decoded = []
for out in output_bits:
    a_decoded.append(int(a_huff_decode[out]))

a_decoded = np.array(a_decoded)

print(a_decoded)

[-26  -3   0  -3  -2  -6   2  -4   1  -4   1   1   5   0   2   0   0  -1
   2   0   0   0   0   0   0  -1   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [11]:
def inverse_zigzag_order(data, rows, cols):
    matrix = np.zeros((rows, cols), dtype=data.dtype)
    index = 0

    for s in range(rows + cols - 1):
        if s % 2 == 0:
            # up-right
            i = min(s, rows - 1)
            j = s - i
            while i >= 0 and j < cols:
                matrix[i][j] = data[index]
                index += 1
                i -= 1
                j += 1
        else:
            # down-left
            j = min(s, cols - 1)
            i = s - j
            while j >= 0 and i < rows:
                matrix[i][j] = data[index]
                index += 1
                i += 1
                j -= 1

    return matrix

# Обратно возвращаемся к 2D представлению
a_decoded = inverse_zigzag_order(a_decoded, 8, 8)
print(a_decoded)


[[-26  -3  -6   2   2   0   0   0]
 [  0  -2  -4   0   0   0   0   0]
 [ -3   1   5  -1  -1   0   0   0]
 [ -4   1   2   0   0   0   0   0]
 [  1   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0]]


In [12]:
# Умножаем на коэффициенты квантования
a_decoded *= y_quant

print(a_decoded)

[[-416  -33  -60   32   48    0    0    0]
 [   0  -24  -56    0    0    0    0    0]
 [ -42   13   80  -24  -40    0    0    0]
 [ -56   17   44    0    0    0    0    0]
 [  18    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0]]


In [13]:
# Обратное ДКП
a_decoded = np.round(scipy.fft.idctn(a_decoded, norm='ortho')).astype(int)

print(a_decoded)

# Сдвигаем обратно в диапазон от 0 до 255
a_decoded += 128
print(a_decoded)

[[-67 -67 -69 -69 -67 -63 -59 -57]
 [-75 -74 -61 -40 -33 -43 -56 -60]
 [-75 -78 -52  -6   8 -20 -51 -59]
 [-63 -77 -53   2  19 -18 -52 -57]
 [-50 -75 -67 -24 -10 -39 -62 -59]
 [-45 -71 -78 -57 -48 -62 -69 -59]
 [-42 -58 -69 -67 -64 -64 -59 -50]
 [-38 -44 -53 -61 -60 -52 -43 -37]]
[[ 61  61  59  59  61  65  69  71]
 [ 53  54  67  88  95  85  72  68]
 [ 53  50  76 122 136 108  77  69]
 [ 65  51  75 130 147 110  76  71]
 [ 78  53  61 104 118  89  66  69]
 [ 83  57  50  71  80  66  59  69]
 [ 86  70  59  61  64  64  69  78]
 [ 90  84  75  67  68  76  85  91]]


In [14]:
# Найдем RMSE
print((np.sum((a_orig - a_decoded)**2)/a_decoded.size)**(0.5))

5.921359641163505
